In [21]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import keras

from keras import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, Flatten, Dense
import tensorflow.keras.datasets as datasets

# A SF Permits Cleaning

### Methode um Y/NaN Spalten zu konvertieren

In [2]:
def replace_y_with_0_1(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Ersetzt Y und leere Felder in Spalten die nur Y und leere Felder erhalten durch 1 und 0 
    :param df: Dataframe in dem sich die Spalten befinden
    :param column: Name der Spalte in der die Werte ersetzt werden sollen
    :return: DataFrame mit modifizierten Spalten
    """
    df[column].fillna(0, inplace=True)
    df[column].replace('Y', 1, inplace=True)
    return df

### Datenset laden

In [3]:
permits_df = pd.read_csv("building_permits.csv")

c:\users\d074009\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# B SF Permits Exploration

# C SF Permits Prediction

### Methode für One-Hot-Encoding

In [4]:
def one_hot_encode(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """
    Erzeugt Dummy-Spalten für jeden Wert in der Quell-Spalte.
    
    Beispiel:
    Wenn eine Tabelle die Spalte 'Klasse' mit 0, 1, 2 als Werte hat, werden die Spalte Klasse 0, Klasse 1 und Klasse 2
    erstellt und die Zeilen der Tabelle an den entsprechenden Stellen auf 0 bzw. 1 gesetzt. 
    :param df: Dataframe in dem die zu ersetzende Spalte ist
    :param col: Spalte die One-Hot-Encoded werden soll
    :return: modifizierter DataFrame
    """
    return df.assign(**{str(col + " " + str(val)): [1 if str(val) in str(cell) else 0 for cell in df[col]] 
                                   for val in df[col].unique()})

# D SF Challenges

# E Neural Networks XOR

![XOR Graph](XOR.png)

w: weight  
b: bias  
t: threshold

Mathematische Funktion:
```
SIGMOID(x) = 1 / (1 + e^(-x))
AND(x, y) = SIGMOID(50x + 50y - 75)
OR(x, y) = SIGMOID(50x + 50y - 25)
NOT(x) = SIGMOID(-50x + 10)

XOR(x, y) = AND(NOT(AND(x, y)), OR(x, y))

XOR(x,y) = SIGMOID(SIGMOID(SIGMOID(50x + 50y - 75) * (-50) + 10) * 50 + SIGMOID(50x + 50y - 25) * 50 - 75)
           \       \       \---------AND---------/             /        \                     /          /
            \       \-------------------NOT-------------------/          \--------OR---------/          /
             \-------------------------------------------AND-------------------------------------------/
```

### Funktionen in Python abbilden und testen

In [5]:
def SIGMOID(x):
    return 1 / (1 + (math.e ** (-x)))

def AND(x, y):
    return SIGMOID(x * 50 + y * 50 - 75)

def OR(x, y):
    return SIGMOID(x * 50 + y * 50 - 25)
    
def NOT(x):
    return SIGMOID(x * (-50) + 10)

def XOR(x, y):
    return AND(NOT(AND(x, y)),OR(x, y))

print(round(XOR(0, 0), 2))
print(round(XOR(0, 1), 2))
print(round(XOR(1, 0), 2))
print(round(XOR(1, 1), 2))

# F Neural Networks Overfitting

### Daten Laden und splitten

In [33]:
cifar10 = datasets.cifar10.load_data()

train, test = cifar10

X_train, Y_train = train
X_test, Y_test = test

Y_train = keras.backend.one_hot(Y_train, 10)
Y_test = keras.backend.one_hot(Y_test, 10)

Y_train = np.reshape(Y_train, (50000, 10))
Y_test = np.reshape(Y_test, (10000, 10))

In [34]:
num_classes = 10

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_25 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_26 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 15, 15, 64)       

In [35]:
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)

model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, batch_size=1000, epochs=10)

Epoch 1/10
50000/50000 [==============================] - 136s 3ms/step - loss: 2.1890 - accuracy: 0.2101
Epoch 2/10
50000/50000 [==============================] - 140s 3ms/step - loss: 1.8715 - accuracy: 0.3344
Epoch 3/10
50000/50000 [==============================] - 151s 3ms/step - loss: 1.6797 - accuracy: 0.4061
Epoch 4/10
50000/50000 [==============================] - 139s 3ms/step - loss: 1.5468 - accuracy: 0.4581
Epoch 5/10
50000/50000 [==============================] - 138s 3ms/step - loss: 1.4507 - accuracy: 0.4938
Epoch 6/10
50000/50000 [==============================] - 137s 3ms/step - loss: 1.3677 - accuracy: 0.5224
Epoch 7/10
41000/50000 [=======================>......] - ETA: 24s - loss: 1.2755 - accuracy: 0.5535

# G Neural Networks Overfitting

# H Feedback